<a href="https://colab.research.google.com/github/ShivamCholin/CS6910_Assignment_1/blob/main/Adam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random          
import numpy as np      
from time import time    

In [ ]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))
def dsigmoid(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [ ]:
def relu(z):
    return np.maximum(z, 0)
def drelu(z):
    return np.heaviside(z, 1)

In [ ]:
def tanh(z):
    return (np.exp(z)-np.exp(-z))/(np.exp(z)+np.exp(-z))
def dtanh(z):
    return 1-tanh(z)**2

In [ ]:
def mbgdlearn(nn, tr_data, epochs, batch_size, learning_rate):
    n = len(tr_data)
    for j in range(epochs):
        time_start=time()
        random.shuffle(tr_data)
        batches = [tr_data[k: k + batch_size] for k in range(0,n,batch_size)]
        for batch in batches:
            mbgd(nn, batch, learning_rate)
        time_end=time()
        print('Epoch {0}:time taken {1} seconds, accuracy {2}%'.format(f'{j + 1:2}',1.0*time_end-time_start, 100.0 * evaluate2(nn, tr_data) / len(tr_data)))


In [ ]:
def mbgd(nn, batch, eta):
    nb = [np.zeros(b.shape) for b in nn.biases]
    nw = [np.zeros(w.shape) for w in nn.weights]
    for x, y in batch:
        dnb, dnw = backward(nn, x, y) 

        nb = [nb + dnb for nb, dnb in zip(nb, dnb)]
        nw = [nw + dnw for nw, dnw in zip(nw, dnw)]


    nn.mw = [nn.beta1w*m + (1.0-nn.beta1w)*nw for m,nw in zip(nn.mw,nw)]
    nn.mb = [nn.beta1b*m + (1.0-nn.beta1b)*nb for m,nb in zip(nn.mb,nb)]
    nn.vw = [nn.beta2w*v + (1.0-nn.beta2w)*np.square(nw) for v,nw in zip(nn.vw,nw)]
    nn.vb = [nn.beta2b*v + (1.0-nn.beta2b)*np.square(nb) for v,nb in zip(nn.vb,nb)]
    nn.beta1_expw *= nn.beta1w
    nn.beta2_expw *= nn.beta2w
    nn.beta1_expb *= nn.beta1b
    nn.beta2_expb *= nn.beta2b
    #nn.weights = [w - (eta ) * nw for w, nw in zip(nn.weights, nw)]
    #nn.biases  = [b - (eta ) * nb for b, nb in zip(nn.biases, nb)]
    nn.weights = [w - eta *(m/(1.0-nn.beta1_expw))/ (np.sqrt(v / (1.0 - nn.beta2_expw)) + nn.epsilon) for w, m,v in zip(nn.weights, nn.mw,nn.vw)]
    nn.biases = [b - eta *(m/(1.0-nn.beta1_expb))/ (np.sqrt(v / (1.0 - nn.beta2_expb)) + nn.epsilon) for b, m,v in zip(nn.biases, nn.mb,nn.vb)]

In [ ]:
def softmax(x):
		e_x = np.exp(x - np.max(x))
		return e_x / np.sum(e_x)

In [ ]:
def dcost(act, y):
    act=softmax(act)
    act= act-y
    return act

In [ ]:
class Network:
    num_layers=0
    biases=[]
    weights=[]
    def __init__(self,nl,x,y,act1,act2):
      self.num_layers=nl
      self.biases=x
      self.weights=y
      self.act=act1
      self.dact=act2
      self.vw=[np.zeros(w.shape) for w in self.weights]
      self.vb=[np.zeros(b.shape) for b in self.biases]
      self.mw=[np.zeros(w.shape) for w in self.weights]
      self.mb=[np.zeros(b.shape) for b in self.biases]
      self.gamma=0.9
      self.epsilon = 1e-8
      self.beta1w = 0.9
      self.beta2w = 0.999
      self.beta1_expw = 1.0
      self.beta2_expw = 1.0
      self.beta1b = 0.9
      self.beta2b = 0.999
      self.beta1_expb = 1.0
      self.beta2_expb = 1.0

def init_network(layers,actfunc):
    if actfunc=="tanh":
      act1=tanh
      act2=dtanh
    elif actfunc=="sigmoid":
      act1=sigmoid
      act2=dsigmoid
    else:
      act1=relu
      act2=drelu
    return Network(len(layers),[np.random.randn(y, 1) for y in layers[1:]],[np.random.randn(y, x) for x, y in zip(layers[:-1], layers[1:])],act1,act2)

In [ ]:
def forward(nn, a):
    for b, w in zip(nn.biases, nn.weights):
        a = nn.act(np.dot(w, a) + b)
    return a

In [ ]:
def evaluate(nn, te_data):
    test_results = [(np.argmax(forward(nn, x)), y) for (x, y) in te_data]
    return sum(int(x == y) for (x, y) in test_results)

In [ ]:
def evaluate2(nn, te_data):
    test_results = [(np.argmax(forward(nn, x)), y) for (x, y) in te_data]
    return sum(int(x == np.argmax(y)) for (x, y) in test_results)

In [ ]:
def backward(nn, x, y):
    nb = [np.zeros(b.shape) for b in nn.biases]
    nw = [np.zeros(w.shape) for w in nn.weights]
    activation = x 
    acts = [x]
    zs = []     

    for b, w in zip(nn.biases, nn.weights):
        z = np.dot(w, activation) + b 
        zs.append(z)        
        activation = nn.act(z)   
        acts.append(activation)
    delta = dcost(acts[-1], y) * nn.dact(zs[-1])
    nb[-1] = delta
    nw[-1] = np.dot(delta, acts[-2].transpose())
    for i in range(2, nn.num_layers):
        z = zs[-i]
        sp = nn.dact(z)
        delta = np.dot(nn.weights[-i + 1].transpose(), delta) * sp
        nb[-i] = delta
        nw[-i] = np.dot(delta, acts[-i - 1].transpose())
    return (nb, nw)

In [ ]:
import keras
def change_y(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e
fashion_mnist = keras.datasets.fashion_mnist
tr_data, te_data = fashion_mnist.load_data()
training_x = [np.reshape(x, (784, 1))/255 for x in tr_data[0]]
training_y = [change_y(y) for y in tr_data[1]]
tr_data = zip(training_x, training_y)
test_inputs = [np.reshape(x, (784, 1))/255 for x in te_data[0]]
te_data = zip(test_inputs, te_data[1])
tr_data=list(tr_data)
te_data=list(te_data)

In [ ]:
nn = init_network([784, 30, 10],"sigmoid")
epochs = 20
batch_size =10
learning_rate = 0.001
print('start')
mbgdlearn(nn, tr_data, epochs, batch_size, learning_rate)
print('accuracy {0}%'.format(100.0 * evaluate(nn, te_data) / len(te_data)))

start


KeyboardInterrupt: ignored